 # Clean data

In [1]:
import pandas as pd
import re
from pprint import pprint
SOURCE_DATA_FILE = "Numbers to parsing.tsv"

In [2]:
source_table = pd.read_csv(SOURCE_DATA_FILE, sep='\t')

In [3]:
source_table.head(5)

,query,alternative query,main passage,citation
0,踢猫效应在直面上如何体现？,踢猫效应最初指的是什么？,一位父亲的同事耽误了他很多 时间，影响了他当天工作的完成。想 着明天要挨上司的责骂，他一肚子...,"李秋生,王小文. 踢猫效应与情绪传递[J]. 大众心理学,2016(10):41-42. \n"
1,解释一下踢猫效应,NaN,个体会对 弱于自己或者等级低于自己的对象 发泄不满情绪，而产生坏情绪传染 的连锁反应。但是这...,"李秋生,王小文. 踢猫效应与情绪传递[J]. 大众心理学,2016(10):41-42. \n"
2,踢猫效应会带来什么连锁反应？,NaN,假如这只猫对我们不重要，而 且它选择了反抗，我们很有可能和 它陷入战争。人踢了猫一下，猫也会...,"李秋生,王小文. 踢猫效应与情绪传递[J]. 大众心理学,2016(10):41-42. \n"
3,根据踢猫效应，踢猫会带来什么后果,踢猫又可能带来的结果有哪些？,假如这只猫对我们不重要，而 且它选择了隐忍，这是最好的情况， 它默默地承受了你的坏心情，没有...,"李秋生,王小文. 踢猫效应与情绪传递[J]. 大众心理学,2016(10):41-42. \n"
4,不踢猫的话，可以更好地发泄负面情绪吗？,有在被人身上发泄情绪的冲动时候可以如何更好的解决问题？,当然是可以的，但是并非简单 粗暴地把猫一脚踢开。我们的家人、 朋友、同事、同学都可以成为这只...,"李秋生,王小文. 踢猫效应与情绪传递[J]. 大众心理学,2016(10):41-42. \n"


In [4]:
# define some stuff for storing queries, passages, and relevance marking
queries = {} # dictionary of 'qid': 'query', ...
passages = {} # dictionary of 'pid': 'passage', ...
citations = {} # dictionary of 'pid': 'citation', ...
matches = [] # dictionary of matching passages entries {'qid': qid, 'pid': pid}
es_df = pd.DataFrame() # this is the table for elastic search index

# numbering counters
qid_counter = 1
pid_counter = 1

for row in source_table.iterrows():

    row_index = row[0]
    row_content = row[1]
    query = str(row_content['query'])
    alt_query = str(row_content['alternative query'])
    passage = str(row_content['main passage'])
    citation = str(row_content['citation'])


    # trim unnecessary spaces and tabs
    query = query.strip()
    alt_query = alt_query.strip()
    passage = passage.strip()
    citation = citation.strip()

    # trim [] from citation
    citation = re.sub(r'\[\d\]', '', citation)

    # strip space in chinese
    passage = re.sub(r' ', '', passage)
    # fix unwanted new lines in chinese
    passage = re.sub(r'(?<![。？！])\n', '', passage)
    # strip numbering at passage beginning
    passage = re.sub(r'^(\d\.)', '', passage)

    # store the entry in dic and record match
    qid = 'q' + str(qid_counter)
    pid = 'p' + str(pid_counter)
    queries[qid] = query
    passages[pid] = passage
    citations[pid] = citation
    matches.append({'qid': qid, 'pid': pid})
    qid_counter += 1
    pid_counter += 1

    if alt_query != "nan":
        # there is an alt query, make extra query
        qid = 'q' + str(qid_counter)
        queries[qid] = alt_query
        matches.append({'qid': qid, 'pid': pid})
        qid_counter += 1
        es_df = es_df.append({'qid': 'q' + str(qid_counter - 2), 'pid': pid, 'query': query, 'passage': passage, 'alt_query': alt_query, 'alt_qid': qid, 'citation': citation}, ignore_index=True)
    else:
        alt_query = None
        es_df = es_df.append({'qid': qid, 'pid': pid, 'query': query, 'passage': passage, 'alt_query': None, 'alt_qid': None, 'citation': citation}, ignore_index=True)

In [5]:
es_df = es_df[['qid', 'query', 'alt_qid', 'alt_query', 'pid', 'passage', 'citation']]

In [6]:
es_df.sample(30)

,qid,query,alt_qid,alt_query,pid,passage,citation
4,q7,不踢猫的话，可以更好地发泄负面情绪吗？,q8,有在被人身上发泄情绪的冲动时候可以如何更好的解决问题？,p5,当然是可以的，但是并非简单粗暴地把猫一脚踢开。我们的家人、朋友、同事、同学都可以成为这只猫，...,"李秋生,王小文. 踢猫效应与情绪传递[J]. 大众心理学,2016(10):41-42."
260,q409,学习可以是父母的挡箭牌吗？,None,None,p261,有学者曾经指出：中国家庭的十种界限不清主要包括“家庭中有一个过度严厉的父亲”、“过分唠叨的妈...,"葛虹宇. 高中生亲子心理边界的调查研究[D].哈尔滨师范大学,2052"
96,q145,改进和增进亲子关系的建议有哪些？,q146,怎样增进亲子关系？,p97,家长对孩子要抱有合理期望。绝大多数家长对孩子都抱有很大期望，希望一代更比一代强。对于受教育程...,"不做焦虑父母，从良好亲子关系开始——疫情期间亲子关系状况调查报告[J].教育家,2021(2..."
229,q378,成长中的边界会带来什么问题？,None,None,p230,有些人起初具有薄弱的自我界线，对自己的冲动与愿望非常开放或恐慌，之后有可能会变得猜忌多疑，出...,"葛虹宇. 高中生亲子心理边界的调查研究[D].哈尔滨师范大学,2021"
227,q376,高中孩子突然的变化有什么？,None,None,p228,究者发现一些孩子没有办法处理好与父母之间的界限，比如说哪些事情是需要父母说了算，哪些事情是孩...,"葛虹宇. 高中生亲子心理边界的调查研究[D].哈尔滨师范大学,2019."
20,q28,家庭对孩子行为习惯的影响,q29,家庭如何影响孩子的行为习惯,p21,SueZupanec等（2010）做出明确，家庭教育、学校与社会环境都会对孩子的学习习惯产生...,"莫申杰. 亲子关系对初中生学习习惯的影响研究[D].湖南农业大学,2020."
22,q31,父母应该如何与孩子沟通？,q32,父母和孩子沟通的方式有哪些？,p23,沟通是相互的过程，是双方彼此互动、彼此影响的作用表现。父母在对子女提出要求的同时，也要接受子...,"莫申杰. 亲子关系对初中生学习习惯的影响研究[D].湖南农业大学,2020."
179,q301,亲子沟通中有什么需要注意的点？,q302,有哪些行为能改善亲子沟通？,p180,不管是对于家长还是孩子，都要注意。1.主动沟通，2.隐私与边界，3.表达关心，4.平等地沟通...,李玲. 初中生亲子沟通、成就动机及其团体辅导的研究 [D]. 西南大学. 2044
30,q43,为什么专制教育模式不好？,q44,专制型家长要避免什么？,p31,在专制型的家庭中，父母与学生的亲子互动是建立在家长权威下的压迫式关系，亲子交流紧张、沟通有障...,"莫申杰. 亲子关系对初中生学习习惯的影响研究[D].湖南农业大学,2020."
73,q111,尊重孩子的重要性,q112,为什么父母要尊重孩子,p74,尊重孩子的话语权是建立亲子关系的纽带。父母绝对不能因为孩子小，认为他们没思想，没意识而忽略他...,甄锦珠. 在倾听中陪伴孩子共同成长之我见[A]. 中国智慧工程研究会智能学习与创新研究工作委...


In [7]:
queries["q13"]

'亲子关系会影响孩子适应社会的能力吗'

In [8]:
citations["p111"]

'胡悦. 亲子沟通与青少年健康成长[D].哈尔滨工程大学,2007.'

In [9]:
len(es_df)

267

In [10]:
len(passages)

267

# Dunk into elastic search

In [11]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch()

In [14]:
# clear index
es.indices.delete(index='ctb-nlp-v1', ignore=[400, 404])

# dump into elasticsearch
temp_bulk_actions = []
for index, row in es_df.iterrows():
	action = {
		"_index": "ctb-nlp-v1",
		"_source": {
			'qid': row['qid'], 
			'query': row['query'], 
			'alt_qid': row['alt_qid'], 
			'alt_query': row['alt_query'], 
			'pid': row['pid'], 
			'passage': row['passage'], 
			'citation': row['citation'],
			}
		}
	temp_bulk_actions.append(action)

helpers.bulk(es, temp_bulk_actions)

(267, [])

In [16]:
# don't know what this does
es.indices.refresh(index="ctb-nlp-v1")

{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

# Now make training data

it's not obvious how to generate a score label for query passage pairs... but what we can do is to generate a pseudo relevalence score using characteristics of the query and the passage.

- Set default for relevant to be 0.8 and score of negative samples to 0.1
- the relevent score shall be 0.65 + 0.30 tanh(0.1 relu(query length - 8)) + 0.05 ((1/512) relu(passage length - 200))
- the irrelevant score shall be 0.22 - 0.10 tanh(0.1 relu(query length - 8)) - 0.03 ((1/512) relu(passage length - 200))

In [24]:
for match in matches:
    matched_qid = match['qid']
    matched_pid = match['pid']
    

q1
q2
q3
q4
q5
q6
q7
q8
q9
q10
q11
q12
q13
q14
q15
q16
q17
q18
q19
q20
q21
q22
q23
q24
q25
q26
q27
q28
q29
q30
q31
q32
q33
q34
q35
q36
q37
q38
q39
q40
q41
q42
q43
q44
q45
q46
q47
q48
q49
q50
q51
q52
q53
q54
q55
q56
q57
q58
q59
q60
q61
q62
q63
q64
q65
q66
q67
q68
q69
q70
q71
q72
q73
q74
q75
q76
q77
q78
q79
q80
q81
q82
q83
q84
q85
q86
q87
q88
q89
q90
q91
q92
q93
q94
q95
q96
q97
q98
q99
q100
q101
q102
q103
q104
q105
q106
q107
q108
q109
q110
q111
q112
q113
q114
q115
q116
q117
q118
q119
q120
q121
q122
q123
q124
q125
q126
q127
q128
q129
q130
q131
q132
q133
q134
q135
q136
q137
q138
q139
q140
q141
q142
q143
q144
q145
q146
q147
q148
q149
q150
q151
q152
q153
q154
q155
q156
q157
q158
q159
q160
q161
q162
q163
q164
q165
q166
q167
q168
q169
q170
q171
q172
q173
q174
q175
q176
q177
q178
q179
q180
q181
q182
q183
q184
q185
q186
q187
q188
q189
q190
q191
q192
q193
q194
q195
q196
q197
q198
q199
q200
q201
q202
q203
q204
q205
q206
q207
q208
q209
q210
q211
q212
q213
q214
q215
q216
q217
q218
q219
q220
q221
q22